# TryCatch in Lettuce

* Author: Spencer Wilson
* Date: March 14 2021
* Intial Purpose: In CSPB3155-Spring2022 questions arose on why the operational semantic rules involving stores still need a store in error cases. In the version of Lettuce of the reference assignment, there was no value to providing these stores, but if we add try/catch/finally statements to the Lettuce language, then these suddently become important. At this time, this is not in scope for the course
* My advanced apologies for any typos

## A Lettuce definition of Try Catch

* Lettuce concrete syntax: try e1 catch e2
* Lettuce abstract syntax as Expr: TryCatch(e1, e2)
* Logic:
    * I'm not skilled in latex, so I'll write this in an alternate form. Let 'C' be Conclusion is what goes below the line of an operational semantic. Let P\<n> be a Premis\<n> are the ordered items above the line.
    * Rule: Try-OK
        * P0: eval(e1, env,store) = (v1, store1)
        * P1: v1 != Error
        * C: eval(TryCatch(e1,e2), env, store) = (v1, store1)
    * Rule: Try-Catch
        * P0: eval(e1, env,store) = (Error, store1)
        * P1: eval(e2, env, store1) = (v2, store2)
        * C: eval(TryCatch(e1,e2), env, store) = (v2, store2)

##  Try Catch in Scala

Is that a reasonable operational semantic? Is that what Scala does? Let's look at a sample

In [ ]:
// Scala Sample
var x = 10
try {
    x = 20
    throw new RuntimeException("foo")
    x = 30
    x
} catch {
    case _RunTimeException => x
}

// The logic is the same as what scala does. The result here is 20, not 10 and not 30.

The logic in our operational semantic rules for Lettuce follows the same logic as Scala. The value of the Scala program above is 20.

## An Alternative Semantic

This is a bit of a side note...

What if instead wanted a semantic where it returned 10 instead? If the try block fails, then the side effects inside of try effectively do not occur. This could be implmented as a semtic for Lettuce. I'm unaware of a langauge that uses this feature or why it would. Spitballing - a specialized database langauge might benefit from this. Here is a reasonable operational semantic rule for that feature

* Rule: Try-OK: remains the same
    * P0: eval(e1, env,store) = (v1, store1)
    * P1: v1 != Error
    * C: eval(TryCatch(e1,e2), env, store) = (v1, store1)
* Rule: Try-Catch
    * P0. eval(e1, env, store) = (Error, store1)
    * C. eval(TryCatch(e1, e2), env, store) = eval(e2, env, store)
        * NOTE: It's not using store 1
        * NOTE: If our language also had I/O like print statements, we may need to be more sophisticated with this rule for it to really take the try as an atomic expression
  

Lettuce is special enough as it exists today, so we will not implement this semantic in the code below.

## Interpreter

In [ ]:
// I've selected a subset of Lettuce to allow us to trigger a few kinds of errors
sealed trait Value
case class NumValue(n:Double) extends Value
case class BoolValue(b:Boolean) extends Value
case class Reference(r:Int) extends Value
case object ErrorValue extends Value

sealed trait Expr
case class Const(n:Double) extends Expr
case class NewRef(e1:Expr) extends Expr
case class Plus(e1:Expr, e2:Expr) extends Expr
case class Geq(e1:Expr, e2:Expr) extends Expr
case class Let(x:String, e1:Expr, e2:Expr) extends Expr
case class Ident(x:String) extends Expr
case class AssignRef(e1:Expr, e2:Expr) extends Expr
case class DeRef(e1:Expr) extends Expr
case class Seq(e1:Expr, e2:Expr) extends Expr
case class TryCatch(e1:Expr, e2:Expr) extends Expr

sealed trait Program
case class TopLevel(e: Expr) extends Program

In [ ]:
// Need to define a store
case class ImmutableStore(val nCells: Int, val storeMap: Map[Int, Value])
def createNewCell(s: ImmutableStore, v: Value): (ImmutableStore, Int) = {
        /*- make a new cell -*/
        val j = s.nCells
        val nMap = s.storeMap + (j -> v)
        val nStore = ImmutableStore(s.nCells + 1, nMap) // Make a new store with one more cell
        (nStore, j)
}
def lookupCellValue(s: ImmutableStore, j: Int): Value = {
        if (s.storeMap.contains(j)){
            s.storeMap(j)
        } else {
            throw new IllegalArgumentException(s"Illegal lookup of nonexistant location $j")
        }
}
def assignToCell(s: ImmutableStore, j: Int, v: Value): ImmutableStore = {
        if (s.storeMap.contains(j)){
            val nMap = s.storeMap + (j -> v) // Update the store map.
            ImmutableStore(s.nCells, nMap)
        } else {
            throw new IllegalArgumentException(s"Illegal assignment to nonexistent location $j")
        }
    }


In [ ]:
// must define the usual interpreter functions

def eval(e:Expr, env:Map[String, Value], sto:ImmutableStore): (Value, ImmutableStore) = e match {
    case Const(n) => (NumValue(n), sto)
    case NewRef(e1) => eval(e1, env, sto) match {
        case r@(ErrorValue, _)  => r
        case (v1, sto1) => {
            val (sto2, ref) = createNewCell(sto1, v1)
            (Reference(ref), sto2)
        }
    }
    case Plus(e1, e2) => eval(e1, env, sto) match {
        case r@(ErrorValue, _)  => r
        case (NumValue(n1), sto1) => eval(e2, env, sto1) match {
            case r@(ErrorValue, _)  => r
            case (NumValue(n2), sto2) => {
                (NumValue(n1 + n2), sto2)
            }
            case (_, sto2) => (ErrorValue, sto2)
        }
        case (_, sto1) => (ErrorValue, sto1)
    }
    case Geq(e1, e2) => eval(e1, env, sto) match {
        case r@(ErrorValue, _)  => r
        case (NumValue(n1), sto1) => eval(e2, env, sto1) match {
            case r@(ErrorValue, _)  => r
            case (NumValue(n2), sto2) => {
                (BoolValue(n1 >= n2), sto2)
            }
            case (_, sto2) => (ErrorValue, sto2)
        }
        case (_, sto1) => (ErrorValue, sto1)
    }
    case Let(x, e1, e2) => eval(e1, env, sto) match {
        case r@(ErrorValue,_) => r
        case (v1, sto1) => eval(e2, env + {x -> v1}, sto1)        
    }
    case Ident(x) => {
        if (env contains x) (env(x), sto)
        else (ErrorValue, sto)
    }
    case AssignRef(e1, e2) => eval(e1, env, sto) match {
        case r@(ErrorValue,_) => r
        case (Reference(ref), sto1) => eval(e2, env, sto) match {
            case r@(ErrorValue,_) => r
            case (v2, sto2) => {
                val stop = assignToCell(sto2, ref, v2)
                (v2, stop)
            }
        }
        case (_, sto1) => (ErrorValue, sto1)
    }
    case DeRef(e1) => eval(e1, env, sto) match {
        case r@(ErrorValue,_) => r
        case (Reference(ref), sto1) => {
            val v1 = lookupCellValue(sto1, ref)
            (v1, sto1)
        }
        case (_, sto1) => (ErrorValue, sto1)
    }
    case TryCatch(e1, e2) => eval(e1, env, sto) match {
        // The op'sem' that we typically see
        case r@(ErrorValue, sto1) => eval(e2, env, sto1)
        case r => r
        // // SAMPLE Solution for the other op'sem' that doesn't work in any lanaguage I know of
        // case r@(ErrorValue, _) => eval(e2, env, sto)
        // case r => r
    }
    case _ => ???
}

// Convenient for top levels. it's a polymorphic definition of the 'eval' function
// I think that Jupyter only allows this if they are in the same box
def eval(p:Program):Value = {
    val TopLevel(e0) = p
    eval(e0, Map(), ImmutableStore(0,Map()))._1
}

## Tests

In [ ]:
// CANNOT compare a boolean to an int - in Lettuce
// let x = NewRef(10) in 
//     try let tmp = assignref(x, 20) in x > 10 > 10 
//     catch deref(x)
val tmp = Let("tmp", AssignRef(Ident("x"), Const(20)), Geq(Geq(Ident("x"), Const(10)), Const(10)))
val tc = TryCatch(tmp, DeRef(Ident("x")))
val x = Let("x", NewRef(Const(10)), tc)
val p = TopLevel(x)
val v = eval(p)

assert(NumValue(20) == v)
println("\n\n\nPASSED THE TEST\n\n\n")

In [ ]:
// CANNOT lookup a variable that doesn't exist
// let x = NewRef(10) in 
//     try let t0 = assignref(x, 20) in let t1 = y in let t2 = assignRef(x,30) in deref(x)
//     catch deref(x)
val t2 = Let("t2", AssignRef(Ident("x"), Const(30)), DeRef(Ident("x")))
val t1 = Let("t1", Ident("y"), t2)
val t0 = Let("t0", AssignRef(Ident("x"), Const(20)), t1)
val tc = TryCatch(tmp, DeRef(Ident("x")))
val x = Let("x", NewRef(Const(10)), tc)
val p = TopLevel(x)
val v = eval(p)

assert(NumValue(20) == v)
println("\n\n\nPASSED THE TEST\n\n\n")
